<a href="https://colab.research.google.com/github/Dohy-Lee/Do_it-BERT_GPT/blob/main/doc_cls_deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ratsnlp
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 582 kB 60.8 MB/s 
     |████████████████████████████████| 2.8 MB 54.3 MB/s 
     |████████████████████████████████| 57 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 54.8 MB/s 
     |████████████████████████████████| 419 kB 62.7 MB/s 
     |████████████████████████████████| 141 kB 75.6 MB/s 
     |████████████████████████████████| 101 kB 12.7 MB/s 
     |████████████████████████████████| 3.3 MB 56.2 MB/s 
     |████████████████████████████████| 880 kB 66.9 MB/s 
     |████████████████████████████████| 96 kB 6.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=dc91ce3397147eb1535975b7a30df31a45f8e97e35f18901f93f83745a03d07a
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Su

## 인퍼런스 설정

In [2]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args=ClassificationDeployArguments(
    pretrained_model_name='beomi/kcbert-base', #doc-cls-train-colab.ipynb에서 적용한 pretrained_model_name
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint -doccls', #doc-cls-train-colab.ipynb에서 파인튜닝한 모델의 체크포인트 저장 위치
    max_seq_length=128#토큰 기준 입력 문장 최대 길이. 디폴트값 128
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint -doccls/epoch=0-val_loss=0.26.ckpt


##토크나이저 로드

In [3]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

##체크포인트 로드

In [4]:
import torch
fine_tuned_model_ckpt=torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu'),
)

##BERT설정 로드

In [5]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)

##BERT모델 초기화

In [6]:
from transformers import BertForSequenceClassification
model=BertForSequenceClassification(pretrained_model_config)

##CheckPoint 주입

In [7]:
model.load_state_dict({k.replace('model.',""):v for k,v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

##평가 모드로 전환

In [8]:
model.eval() #드롭아웃 등 학습 때만 사용하는 기법들 무시

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

##인퍼런스 함수

In [13]:
def inference_fn(sentence):
  inputs=tokenizer( #문장을 토큰화한 뒤에 input_id, attention_mask, token_type_ids 만듦
      [sentence],
      max_length=args.max_seq_length,
      padding='max_length',
      truncation=True,
  )
  with torch.no_grad():
    outputs=model(**{k:torch.tensor(v) for k,v in inputs.items()}) #inputs를  파이토치 텐서로 바꾼 후, 모델 계산
    prob=outputs.logits.softmax(dim=1) #logits에 소프트맥스 취하기
    positive_prob=round(prob[0][1].item(),4) #긍정/부정 확률을 소수점 4자리로 반올림
    negative_prob=round(prob[0][0].item(),4)
    pred="긍정(Positive)" if torch.argmax(prob)==1 else "부정(Negative)" #예측 확률의 최댓값 위치에 따라 pred 만들기
  return{
      'sentence':sentence,
      'prediction':pred,
      'positive_data':f"긍정 {positive_prob}",
      'negative_data':f"부정 {negative_prob}",
      'positive_width':f"{positive_prob*100}%",
      'negative_width':f"{negative_prob*100}%",
  }

##웹 서비스 시작

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [10]:
!mkdir /root/.ngrok2 && echo "authtoken: {개인토큰키}" > /root/.ngrok2/ngrok.yml

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app=get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d524-35-240-222-84.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:55:56] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:55:57] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:02] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:09] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:16] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:21] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:24] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:33] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:38] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:42] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:45] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:48] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Aug/2022 05:56:54] "POST /api